<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#cons-of-&quot;one-representation&quot;-of-words" data-toc-modified-id="cons-of-&quot;one-representation&quot;-of-words-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>cons of "one representation" of words</a></span></li><li><span><a href="#Contextual-word-embeddings-built-from-natural-machine-translation-(CoVe)" data-toc-modified-id="Contextual-word-embeddings-built-from-natural-machine-translation-(CoVe)-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Contextual word embeddings built from natural machine translation (CoVe)</a></span></li><li><span><a href="#Tag-LM-(pre-ELMO)-for-Name-Entity-Recognition-(NER)" data-toc-modified-id="Tag-LM-(pre-ELMO)-for-Name-Entity-Recognition-(NER)-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Tag LM (pre ELMO) for Name Entity Recognition (NER)</a></span></li><li><span><a href="#ELMO" data-toc-modified-id="ELMO-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>ELMO</a></span><ul class="toc-item"><li><span><a href="#weighting-of-layers-for-different-tasks" data-toc-modified-id="weighting-of-layers-for-different-tasks-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>weighting of layers for different tasks</a></span></li><li><span><a href="#Bidirectional-LSTM-language-model" data-toc-modified-id="Bidirectional-LSTM-language-model-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Bidirectional LSTM language model</a></span></li><li><span><a href="#Word-embeddings-built-by-ELMO" data-toc-modified-id="Word-embeddings-built-by-ELMO-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Word embeddings built by ELMO</a></span></li></ul></li><li><span><a href="#Transformer" data-toc-modified-id="Transformer-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Transformer</a></span></li><li><span><a href="#Self-attention" data-toc-modified-id="Self-attention-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Self-attention</a></span><ul class="toc-item"><li><span><a href="#Why-do-we-need-to-scale-by-sqrt(dk)" data-toc-modified-id="Why-do-we-need-to-scale-by-sqrt(dk)-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Why do we need to scale by sqrt(dk)</a></span></li><li><span><a href="#Review-definition-of-Attention" data-toc-modified-id="Review-definition-of-Attention-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Review definition of Attention</a></span></li></ul></li><li><span><a href="#Transformer's-Encoder" data-toc-modified-id="Transformer's-Encoder-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Transformer's Encoder</a></span><ul class="toc-item"><li><span><a href="#Self-attention-in-the-encoder" data-toc-modified-id="Self-attention-in-the-encoder-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Self-attention in the <strong>encoder</strong></a></span></li><li><span><a href="#Multihead-attention" data-toc-modified-id="Multihead-attention-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Multihead attention</a></span></li><li><span><a href="#Why-we-need-to-do-a-linear-tfm-for-Q,K,V-(e.g.-Q@Wi)-before-doing-attention?" data-toc-modified-id="Why-we-need-to-do-a-linear-tfm-for-Q,K,V-(e.g.-Q@Wi)-before-doing-attention?-7.3"><span class="toc-item-num">7.3&nbsp;&nbsp;</span>Why we need to do a linear tfm for Q,K,V (e.g. Q@Wi) before doing attention?</a></span></li><li><span><a href="#Complete-transformer-block" data-toc-modified-id="Complete-transformer-block-7.4"><span class="toc-item-num">7.4&nbsp;&nbsp;</span>Complete transformer block</a></span></li><li><span><a href="#Residual-layer" data-toc-modified-id="Residual-layer-7.5"><span class="toc-item-num">7.5&nbsp;&nbsp;</span>Residual layer</a></span></li><li><span><a href="#Fully-connected-feed-forward-network" data-toc-modified-id="Fully-connected-feed-forward-network-7.6"><span class="toc-item-num">7.6&nbsp;&nbsp;</span>Fully connected feed-forward network</a></span></li><li><span><a href="#Layer-norm" data-toc-modified-id="Layer-norm-7.7"><span class="toc-item-num">7.7&nbsp;&nbsp;</span>Layer norm</a></span></li><li><span><a href="#Multi-block-with-multi-head-attention-vs-LSTM/RNN" data-toc-modified-id="Multi-block-with-multi-head-attention-vs-LSTM/RNN-7.8"><span class="toc-item-num">7.8&nbsp;&nbsp;</span>Multi-block with multi-head attention vs LSTM/RNN</a></span></li><li><span><a href="#The-complete-encoder" data-toc-modified-id="The-complete-encoder-7.9"><span class="toc-item-num">7.9&nbsp;&nbsp;</span>The complete encoder</a></span></li><li><span><a href="#Token-embeddings-and-tie-weight-(weight-sharing)" data-toc-modified-id="Token-embeddings-and-tie-weight-(weight-sharing)-7.10"><span class="toc-item-num">7.10&nbsp;&nbsp;</span>Token embeddings and tie-weight (weight sharing)</a></span></li><li><span><a href="#Positional-encoding" data-toc-modified-id="Positional-encoding-7.11"><span class="toc-item-num">7.11&nbsp;&nbsp;</span>Positional encoding</a></span><ul class="toc-item"><li><span><a href="#Relative-vs-Absolute-Position" data-toc-modified-id="Relative-vs-Absolute-Position-7.11.1"><span class="toc-item-num">7.11.1&nbsp;&nbsp;</span>Relative vs Absolute Position</a></span></li><li><span><a href="#Learnable-positional-encoding" data-toc-modified-id="Learnable-positional-encoding-7.11.2"><span class="toc-item-num">7.11.2&nbsp;&nbsp;</span>Learnable positional encoding</a></span></li><li><span><a href="#Function-based-positional-encoding" data-toc-modified-id="Function-based-positional-encoding-7.11.3"><span class="toc-item-num">7.11.3&nbsp;&nbsp;</span>Function-based positional encoding</a></span></li></ul></li><li><span><a href="#BERT-vs-GPT-in-term-of-positional-encoding" data-toc-modified-id="BERT-vs-GPT-in-term-of-positional-encoding-7.12"><span class="toc-item-num">7.12&nbsp;&nbsp;</span>BERT vs GPT in term of positional encoding</a></span></li><li><span><a href="#TODO:-visualization-in-encoder" data-toc-modified-id="TODO:-visualization-in-encoder-7.13"><span class="toc-item-num">7.13&nbsp;&nbsp;</span>TODO: visualization in encoder</a></span></li></ul></li><li><span><a href="#Transformer's-Decoder" data-toc-modified-id="Transformer's-Decoder-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Transformer's Decoder</a></span><ul class="toc-item"><li><span><a href="#Output-of-Decoder-for-training" data-toc-modified-id="Output-of-Decoder-for-training-8.1"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>Output of Decoder for training</a></span></li><li><span><a href="#Masks" data-toc-modified-id="Masks-8.2"><span class="toc-item-num">8.2&nbsp;&nbsp;</span>Masks</a></span></li><li><span><a href="#Output-of-Decoder-for-Inference" data-toc-modified-id="Output-of-Decoder-for-Inference-8.3"><span class="toc-item-num">8.3&nbsp;&nbsp;</span>Output of Decoder for Inference</a></span></li></ul></li><li><span><a href="#Dropout" data-toc-modified-id="Dropout-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Dropout</a></span></li><li><span><a href="#Full-model" data-toc-modified-id="Full-model-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Full model</a></span></li><li><span><a href="#OpenAI-GPT" data-toc-modified-id="OpenAI-GPT-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>OpenAI GPT</a></span><ul class="toc-item"><li><span><a href="#Plugging-the-classification-head" data-toc-modified-id="Plugging-the-classification-head-11.1"><span class="toc-item-num">11.1&nbsp;&nbsp;</span>Plugging the classification head</a></span></li><li><span><a href="#Other-tasks?" data-toc-modified-id="Other-tasks?-11.2"><span class="toc-item-num">11.2&nbsp;&nbsp;</span>Other tasks?</a></span></li></ul></li><li><span><a href="#BERT" data-toc-modified-id="BERT-12"><span class="toc-item-num">12&nbsp;&nbsp;</span>BERT</a></span><ul class="toc-item"><li><span><a href="#BERT-next-sentence-prediction" data-toc-modified-id="BERT-next-sentence-prediction-12.1"><span class="toc-item-num">12.1&nbsp;&nbsp;</span>BERT next sentence prediction</a></span></li><li><span><a href="#a-typical-embedding-input-of-BERT" data-toc-modified-id="a-typical-embedding-input-of-BERT-12.2"><span class="toc-item-num">12.2&nbsp;&nbsp;</span>a typical embedding input of BERT</a></span></li><li><span><a href="#losses" data-toc-modified-id="losses-12.3"><span class="toc-item-num">12.3&nbsp;&nbsp;</span>losses</a></span></li><li><span><a href="#Plugging-the-classification-head" data-toc-modified-id="Plugging-the-classification-head-12.4"><span class="toc-item-num">12.4&nbsp;&nbsp;</span>Plugging the classification head</a></span></li><li><span><a href="#BERT-model-architecture-and-training" data-toc-modified-id="BERT-model-architecture-and-training-12.5"><span class="toc-item-num">12.5&nbsp;&nbsp;</span>BERT model architecture and training</a></span></li><li><span><a href="#BERT-model-finetuning" data-toc-modified-id="BERT-model-finetuning-12.6"><span class="toc-item-num">12.6&nbsp;&nbsp;</span>BERT model finetuning</a></span></li></ul></li><li><span><a href="#AlBERT" data-toc-modified-id="AlBERT-13"><span class="toc-item-num">13&nbsp;&nbsp;</span>AlBERT</a></span></li></ul></div>

# cons of "one representation" of words

Up until now, we’ve basically had one representation of words: The word vectors that we learned about at the beginning
- Word2vec, GloVe, fastText
    
Problems:
- **Problem 1**: a word can have different meanings ("sense"), depending on the context, and now we try to collapse all the meanings of a word into 1 single vector, and hope that your model is complex enough that it can pick out the correct word meaning
- quick solution: define different word-senses for each word and build a vector for each

=> what you want is to **build not a perfect word vector that can capture all of its meanings, but a word vector that is correct in your given context**

- **Problem 2**: on a more general view, not just different meanings but words have different aspects, including semantics, syntactic behavior, and register/connotations.
    

However, these problems have been solved by the process of building a language model that predicts next word; **by doing so you already generate a context-specific representation of words**

# Contextual word embeddings built from natural machine translation (CoVe)

Reminder:

![](s4/translation_attention_archi.jpg)

The limitation of CoVe is obvious: 
- (1) **pre-training is bounded by available datasets on the supervised translation task**;
    - Since encoder and decoder are trained end-to-end at the same time, encoder only uses dataset from translation task to update weight embedding (as opposed to be trained separately on bigger dataset such as WikiText - as seen in ULMFIT and advanced tranformer architecture
- (2) the contribution of CoVe to the final performance is constrained by the task-specific model architecture (as each task requires a specific model).
    - **The power of CoVe word embedding is constrained by the limited choices of model architecture and the limited power of task-specific model. At this point we only have LSTM model (with attention perhaps)** with few layer deep, unlike stacking Transformers that we will see soon
        - **We need a model architecture that can train word embedding and that same model can be finetuned on numerous task**
        - **We need a model architecture that is good for every task to take away this task-dependency**, similar to a CNN model with replacable head for tasks. This is what BERT or OpenAI-GPT can do

# Tag LM (pre ELMO) for Name Entity Recognition (NER)

![](images/context_1.png)

Note that "word embedding model" is the word embedding from w2v/glove/fasttext

![](images/context_2.png)

Token embedding is from w2v/glove/fasttext

Note: 
- only concat the hidden state, not the embeddings of LM model and w2v model
- This is **not end-to-end training**: by 'pretrained', we only use the output hidden state of LM model as input for the Sequence-tagging feature. **In another word, the LM model is trained first ('pretrained') and is frozen after we extract the hidden states**


# ELMO

Peters et al. (2018): ELMo: Embeddings from Language
Models
- Train a bidirectional LM
- Aim at performant but not overly large LM:
    - Use 2 biLSTM layers
    - Use character CNN to build initial word representation (only)
        - 2048 char n-gram filters and 2 highway layers, 512 dim projection
    - User 4096 dim hidden/cell LSTM states with 512 dim projections to next input
    - Use a residual connection
    - Tie parameters of token input and output (softmax) and tie these between forward and backward LMs
    
    
    
- TagLM uses only the top layer of LSTM stack; **ELMO utilizes all the layers of LSTM stack, and assign a learnable scaling factor to determine how much of a layer (which can contribute to a "task") to take in**
    - First run biLM to get representations for each word
    - Then let (whatever) end-task model use them
        - Freeze weights of ELMo for purposes of supervised model
        - Concatenate ELMo weights into task-specific model
        - Details depend on task
            - Concatenating into intermediate layer as for TagLM is typical
            - Can provide ELMo representations again when producing outputs, as in a question answering system

## weighting of layers for different tasks

The two biLSTM NLM layers have differentiated uses/meanings
- Lower layer is better for lower-level syntax, etc.
    - Part-of-speech tagging, syntactic dependencies, NER
- Higher layer is better for higher-level semantics
    - Sentiment, Semantic role labeling, question answering, SNLI

## Bidirectional LSTM language model

https://lilianweng.github.io/lil-log/2019/01/31/generalized-language-models.html#bidirectional-language-model

![](images/bidirectional_lstm1.png)

https://medium.com/@plusepsilon/the-bidirectional-language-model-1f3961d1fb27

![](https://miro.medium.com/max/700/1*PB5HXSXOGJrnDa_Si4nj9Q.png)

## Word embeddings built by ELMO

Elmo presents a word as a **linear combination of corresponding hidden layers and the original embedding**

Therefore, length of word embedding should be N times length of a hidden state. N is number of time we stack the LSTM (N=2 in this example)

![](images/elmo.png)

# Transformer

http://nlp.seas.harvard.edu/2018/04/03/attention.html

We want parallelization but RNNs are inherently sequential
- Despite GRUs and LSTMs, **RNNs still need attention mechanism to deal with long range dependencies** – path length between states grows with sequence otherwise
- But if attention gives us access to any state… **maybe we can just use attention and don’t need the RNN?**


![](images/transformer_1.png)

# Self-attention

![](images/transformer_2.png)

![](images/transformer_3.png)

![](images/transformer_12.png)

![](images/selfattention_1.png)

We want to re-represent the word 'represent' in the above picture
- this word is our 'query'
- query can just initially be word-embedding
- **we attend (do dot product) this with all its neighbors** (let's,this,sentence). **Its neighbors are matrix K**, and this step is known as q@K or Q@KT (for the whole matrix Q). This produce attention score
- (scale) and then softmax to have attention distribution (attention probability). 
- then multiply this distribution to **the value (V) which is just the neighborhood itself** and add them, aka produce a weighted combination of your entire neighborhood
- then you can add a feed-forward network to compute new features

=> **Definition of Self-attention**: you **re-express yourself based on the similarity b/t you and your neighbors, by doing a weight combination (dot product) of your entire neighborhood**

**important note**: attention (and self-attention) in general is **permutation invariant: switching orders of words and the results are still the same after applying attention**

(so to maintain order we need positional encoding. See more below)

## Why do we need to scale by sqrt(dk)


![](images/scale_dk.png)

In [19]:
import numpy as np

In [20]:
Q = np.random.randn(100, 1024)
Kt = np.random.randn(1024,100)

In [21]:
np.mean(Q),np.std(Q)

(0.004626749610213863, 1.0032595370128088)

In [22]:
tmp = Q@Kt

In [23]:
np.mean(tmp),np.var(tmp),np.std(tmp)

(0.588758505286575, 998.3248980019707, 31.596279812692675)

## Review definition of Attention

- On each step of the decoder, use **direct connection to the encoder** to **focus on particular part** of **source sentence**
- More formal: a **content-based, memory retrieval mechanism**
    - You have a vector in decoder to attend to all this content ( representations/hidden states) from your encoder to decide what information to absorb, **based on how similar this content is to every position in the memory**
    
![](images/qkv_in_attention_rnn.png)

# Transformer's Encoder

## Self-attention in the **encoder**

- The input word vectors are the queries, keys and values
- In other words: the word vectors themselves select each other
- Word vector stack = Q = K = V
- We’ll see in the decoder why we separate them in the definition

In a self-attention layer all of the keys, values and queries come from the same place, in this case, the output of the previous layer in the encode

## Multihead attention

![](images/transformer_4.png)

**Why?**

By mapping Q,K,V to a lower dimensional space (for each head) using linear tfm, **each head can represent different feature space (syntax, nouns ...) so the attention result can be a MIXING PROPORTIONS of different type of information for each head**

For the e.g below, we have 3 heads, and imagine green arrow captures "who" feature, red captures "did what" feature and purpil captures "to whom". In a sense, **this technique simulate CNN kernel with multiple filters that can filter out different information with each of its filter**

![](images/selfattention_3.png)

Note: mapping Q,K,V to multiple **lower dimensional spaces** using **fully-connected linear layer**
- for head 1 (i=1), use weight matrix WiQ for Q, W1K for K and W1V for V

$$
W^Q_i \in
\mathbb{R}^{d_{\text{model}} \times d_k},
W^K_i \in
\mathbb{R}^{d_{\text{model}} \times d_k},
W^V_i \in
\mathbb{R}^{d_{\text{model}} \times d_v}, 
W^O \in \mathbb{R}^{hd_v \times
d_{\text{model}}}
$$

$$
d_k=d_v=d_{\text{model}}/h=64
$$

- Instead of doing this for the entire columns of Q,K,V
$$
\mathrm{Attention}(Q, K, V) = \mathrm{softmax}(\frac{QK^T}{\sqrt{d_k}})V
$$
we will chop each matrix with originally d_model columns into h sub-matrices by doing matmul with h weight matrices as stated above

In practices, for each Q,K and V, we combine h weight matrices into 1 matrix with shape (d_model,d_model) to optimize calculation **(see graph below)**

## Why we need to do a linear tfm for Q,K,V (e.g. Q@Wi) before doing attention?

![](images/selfattention_2.png)

(at the lower left corner)

**Linear tfm used here can be thought as to generate features**: you **project these vectors into a space using learnable parameters**, so that when dot product is applied, IT CAN BE A **GOOD PROXY FOR SIMILARITY**

![](images/multihead2.jpg)

![](images/multihead1.jpg)

In [1]:
def attention(query, key, value, mask=None, dropout=None):
    """
    Compute 'Scaled Dot Product Attention'
    query: shape (bs,h, n, d_k) as n is # of rows in 1 batch of "query", h is number of heads for attention
    key: shape (bs,h, m, d_k) as m is # of rows in 1 batch of "key"
    value: shape (bs, h, m, d_k)
    
    Note that n = m, and n can be sent_length (number of words for each batch)
    """ 

    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    # (bs,h, n, d_k) @ (bs,h, m, d_k)^T = (bs,h,n,m) aka QKt
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9) # fill mask before softmax
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn # (bs,h,n,m) @ (bs, h, m, d_k) = (bs,h,n,d_k)

In [ ]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4) # 4 linear layers
        # we use the first 3 linears (Wq,Wk and Wv) to calculate attention,
        # and the last linear (Wo) to project attention 
        
        # note that technically Wq (or Wk,Wv) is a combination of 
        # h sub matrices shape (d_model,d_k) for projection to h lower dimensional space

        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key, value, mask=None):
        """
        Implements Figure 2
        
        query: (bs,n,d_model)
        key: (bs,m,d_model)
        value: (bs,m,d_model)
        
        Note that d_model = h * d_k
        and d_k = d_v 
        Also n = m
        n can be sent_length (number of words for each batch)
        """
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)
        
        # 1) Do all the linear projections in batch from d_model => h x d_k 
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]
        # for query: (bs,n,d_model)@(d_model,d_model) = (bs,n,d_model) = (bs,n,h,d_k) = (bs,h,n,d_k)
        # do the same for key and value
        
        # 2) Apply attention on all the projected vectors in batch. 
        x, self.attn = attention(query, key, value, mask=mask, 
                                 dropout=self.dropout)
        # x has shape (bs,h,n,d_k)
        
        # 3) "Concat" using a view and apply a final linear. 
        x = x.transpose(1, 2).contiguous().view(nbatches, -1, self.h * self.d_k)
        # (bs,h,n,d_k) = (bs,n,h,d_k) = (bs,n, h*d_k) aka (bs,n,d_model) then @ (d_model,d_model)
        return self.linears[-1](x)

## Complete transformer block

![](images/transformer_5.png)

In [ ]:
class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        """
        x shape: (bs,n,d_model)
        n can be sent_length (number of words for each batch)
        """

        return x + self.dropout(sublayer(self.norm(x))) 
        # apply residual to multiheaded attention or fully connected NNs based on 'sublayer'
        # Note: Residuals carry positional information to higher layers, among other information

In [ ]:
class EncoderLayer(nn.Module):
    "Encoder is made up of self-attn and feed forward (defined below)"
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn # which is MultiHeadedAttention as defined above
        self.feed_forward = feed_forward # two linear transformations with a ReLU activation in between. See below
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        "Follow Figure 1 (left) for connections."
        """
        x shape: (bs,n,d_model)
        n can be sent_length (number of words for each batch)
        """
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        # In a self-attention layer all of the keys, values and queries come from the same place, 
        # in this case, the output of the previous layer in the encoder
        return self.sublayer[1](x, self.feed_forward) 

## Residual layer

use residuals layer (skip connection) to not only make gradient flow better (avoid vanishing gradients), also to keep positional information from lower layer to high layer)

![](images/residual.png)

## Fully connected feed-forward network

Each of the layers in our encoder and decoder contains a fully connected feed-forward network, which is applied to each position separately and identically. This consists of two linear transformations with a ReLU activation in between.

In [ ]:
class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."
    def __init__(self, d_model=512, d_ff=2048, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

## Layer norm

Why can't we use batchnorm:
- When calculating mean and std of a batch in batchnorm, we assume that item in batch has the same size (e.g. img with same h,w,channels). 
- In RNN, sentences in a batch can have different length (Although this can be fixed with padding, but by adding padding, it can alter the true meaning of mean and std of that batch)

![](images/layernorm_rnn.png)

Layer norm formula is essentially the same as batchnorm, but
- calculated for EACH TRAINING POINT
- No moving average
- Average over the hidden dimension to make the norm strategy independent of the batch size

Layernorm from scratch

In [1]:
import torch

In [2]:
inp = torch.randint(5,(2, 3, 4), dtype = torch.float32)
# bs = 2, seq_len = 3, d_model = 4
inp

tensor([[[4., 0., 4., 2.],
         [0., 1., 1., 1.],
         [0., 0., 4., 4.]],

        [[0., 1., 1., 2.],
         [0., 0., 2., 3.],
         [3., 2., 0., 3.]]])

In [4]:
torch.sum(inp,-1)/ 4

tensor([[2.5000, 0.7500, 2.0000],
        [1.0000, 1.2500, 2.0000]])

In [6]:
tmp_mean = torch.mean(inp,-1,keepdims=True)
tmp_mean

tensor([[[2.5000],
         [0.7500],
         [2.0000]],

        [[1.0000],
         [1.2500],
         [2.0000]]])

In [7]:
tmp_var=torch.square(inp-tmp_mean).mean(-1,keepdim=True)
tmp_var

tensor([[[2.7500],
         [0.1875],
         [4.0000]],

        [[0.5000],
         [1.6875],
         [1.5000]]])

In [11]:
# note that torch.var calculate this slightly different. To get the right result, dont use torch.var
torch.var(inp,-1,keepdim=True)

tensor([[[3.6667],
         [0.2500],
         [5.3333]],

        [[0.6667],
         [2.2500],
         [2.0000]]])

In [9]:
(inp - tmp_mean) / torch.sqrt(tmp_var + 0)

tensor([[[ 0.9045, -1.5076,  0.9045, -0.3015],
         [-1.7321,  0.5774,  0.5774,  0.5774],
         [-1.0000, -1.0000,  1.0000,  1.0000]],

        [[-1.4142,  0.0000,  0.0000,  1.4142],
         [-0.9623, -0.9623,  0.5774,  1.3472],
         [ 0.8165,  0.0000, -1.6330,  0.8165]]])

In [10]:
m = torch.nn.LayerNorm(inp.shape[-1], elementwise_affine=False,eps=0)
output = m(inp)
output

tensor([[[ 0.9045, -1.5076,  0.9045, -0.3015],
         [-1.7321,  0.5774,  0.5774,  0.5774],
         [-1.0000, -1.0000,  1.0000,  1.0000]],

        [[-1.4142,  0.0000,  0.0000,  1.4142],
         [-0.9623, -0.9623,  0.5774,  1.3472],
         [ 0.8165,  0.0000, -1.6330,  0.8165]]])

In [ ]:
class LayerNorm(nn.Module):
    "Construct a layernorm module (See citation for details)."
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        """
        x shape: (bs,n,d_model)
        n can be sent_length (number of words for each batch)
        """
        mean = x.mean(-1, keepdim=True) # taking the mean of each word embedding (?) individually, each word is just a vector
        # similar to layer norm in CV: m = x.mean((1,2,3), keepdim=True), each image is a 3D matrix
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

## Multi-block with multi-head attention vs LSTM/RNN

- RNN:
    - \+ get recurrent information carried in a sentence by feeding 1 batch of words at a time
    - \- CANNOT parallelize using GPU, aka cannot feed the entire batch of sentences at once
- multi-block with multi-head attention
    - \+ With block stacking on top of each other, each block will try to capture information in a chain: 1st block can capture 1st info of a chain, 2nd block can capture 2nd info of a chain => similar to RNN
    - \+ CAN feed the entire batch of sequences => parallelize well using GPUs
    - \- disregard the position of words in a sentence: 2 of the same words in different position might be treated the same way. Also, without position, you dont know where you are at in the sentence
        - Solution: **positional encoding** so same words at different locations have different overall representations:

## The complete encoder

![](images/transformer_6.png)

## Token embeddings and tie-weight (weight sharing)

- Similarly to other sequence transduction models, we use learned embeddings to convert the **input tokens** (of encoder) and **output tokens** (of decoder) to **vectors of dimension  d_model**
- We also use the usual learned linear transformation and softmax function to convert the decoder output to predicted next-token probabilities. 
- In our model, **we share the same weight matrix between the encoder embedding layer and the pre-softmax linear transformation**
    - Sometimes they also share weight between encoder embedding layer and decoder embedding layer (assuming same language input in encoder and decoder, or 2 languages are very similar (**subword tokens in English and French as the 2 languages share over 85k subtokens)

In AWD-LSTM language model, tie_weights will make embedding weight == last layer linear weight. 

```python
['0.encoder.weight: torch.Size([17280, 400])', # this one (word embedding matrix) ... 
 '0.encoder_dp.emb.weight: torch.Size([17280, 400])',
 '0.rnns.0.weight_hh_l0_raw: torch.Size([4608, 1152])',
 '0.rnns.0.module.weight_ih_l0: torch.Size([4608, 400])',
 '0.rnns.0.module.bias_ih_l0: torch.Size([4608])',
 '0.rnns.0.module.bias_hh_l0: torch.Size([4608])',
 '0.rnns.1.weight_hh_l0_raw: torch.Size([4608, 1152])',
 '0.rnns.1.module.weight_ih_l0: torch.Size([4608, 1152])',
 '0.rnns.1.module.bias_ih_l0: torch.Size([4608])',
 '0.rnns.1.module.bias_hh_l0: torch.Size([4608])',
 '0.rnns.2.weight_hh_l0_raw: torch.Size([1600, 400])',
 '0.rnns.2.module.weight_ih_l0: torch.Size([1600, 1152])',
 '0.rnns.2.module.bias_ih_l0: torch.Size([1600])',
 '0.rnns.2.module.bias_hh_l0: torch.Size([1600])',
 '1.decoder.weight: torch.Size([17280, 400])', # ... is the same with this one (last NN layer to predict next word)
 '1.decoder.bias: torch.Size([17280])']
```

In [ ]:
# see code in Full Model section
model.src_embed[0].lut.weight = model.tgt_embed[0].lut.weight
model.generator.lut.weight = model.tgt_embed[0].lut.weight

## Positional encoding

Based on this: https://arxiv.org/pdf/1705.03122.pdf

### Relative vs Absolute Position

- Absolute position: position of words w.r.t to the start of the sentence
- Relative position: position of words w.r.t to some chosen word in the sentence

### Learnable positional encoding

- Learned just like word embedding matrix: We will create a matrix of shape ```[sentence_length,vector_size]```. In short, each positon (position of 1st word, position of 2nd word ...) will be presented by a vector
- BERT use learned positional encoding

### Function-based positional encoding

- in order for the model to make use of the order of the sequence, we must inject some information about the relative or absolute position of the tokens in the sequence
- The positional encodings have the same dimension d_model as the embeddings, so that the two can be summed
- See more at http://nlp.seas.harvard.edu/2018/04/03/attention.html and https://pbcquoc.github.io/transformer/?fbclid=IwAR0UCWWbGgnbaETzfih-iEhsL2KHSszsSTb67mcyBxOzgMpmkSnErOljDYQ

![](images/selfattention_4.png)

This mean the first few dimension will oscillate faster than later dimension (as seen below: dimension 6 is wider than 4 (for even dimension using sin) and also wider than 5; dimension 7 is wider than 5 (for odd dimension using cosine) and also wider than 4 and 6

Think of it like 4 bit binary number 
- 0: 0 0 0 0
- 1: 0 0 0 1
- 2: 0 0 1 0
- 3: 0 0 1 1
- 4: 0 1 0 0
- ...
YOu can see the first bit (from right) oscillates faster than 2nd bit, 2nd bit is faster than 3rd bit...

This will give a sense of order and distance between word vectors

![](images/selfattention_5.png)

![](images/signusoid2.png)

Create a matrix to show distances between word emb vectors after adding the sinusoid function (for first row aka word #0, color is lighter aka distance is bigger when going right)

![](images/selfattention_6.png)

![](images/signusoid.png)

Note for above graph:
- Only the first few dimension of embedding vector represents position (after 200, not much differences) => therefore, **adding positional encoding to word embedding (as opposed to concat them) don't really alter word embedding that much**

- Also, beside capture **absolute position**, Sinusoid (using **both cosine and sine**) allows model to capture **relative position** as well
    - Proof: https://kazemnejad.com/blog/transformer_architecture_positional_encoding/

In [ ]:
class PositionalEncoding(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], 
                         requires_grad=False)
        return self.dropout(x)

## BERT vs GPT in term of positional encoding

- BERT (using **encoder transformer**) doesn't learn much absolute position
    - Why? BERT use **masked language modeling** (predict the masked word using its neighbors), therefore the relative position (of the neighbor words) are more important than absolute position
    
    
- GPT (using **decoder transformer**) actually cares about absolute position more than BERT
    - Why? GPT **predicts the last word each time**, so it cares more about its absolute position
    
Reference: https://arxiv.org/pdf/2010.04903.pdf

## TODO: visualization in encoder

Attention visualization in layer 5

![](images/transformer_7.png)

# Transformer's Decoder

![](images/transformer_11.png)

In [2]:
import torch
import numpy as np

In [6]:
def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    print(subsequent_mask)
    return torch.from_numpy(subsequent_mask) == 0

In [9]:
temp = subsequent_mask(5)

[[[0 1 1 1 1]
  [0 0 1 1 1]
  [0 0 0 1 1]
  [0 0 0 0 1]
  [0 0 0 0 0]]]


In [10]:
temp,temp.shape # mask is at False

(tensor([[[ True, False, False, False, False],
          [ True,  True, False, False, False],
          [ True,  True,  True, False, False],
          [ True,  True,  True,  True, False],
          [ True,  True,  True,  True,  True]]]),
 torch.Size([1, 5, 5]))

In [ ]:
class DecoderLayer(nn.Module):
    "Decoder is made of self-attn, src-attn, and feed forward (defined below)"
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)
 
    def forward(self, x, memory, src_mask, tgt_mask):
        "Follow Figure 1 (right) for connections."
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask)) 
        # key and value (m) from encoder
        return self.sublayer[2](x, self.feed_forward)

**One problem with using this Decoder for seq2seq task such as neural machine translation** is it will just be similar to LSTM/RNN (autoregressive model as you use observations from previous time steps as input to predict the value at the next time step)

## Output of Decoder for training

Using teacher forcing: input the **entire** sentence (that needs to be translated), and decoder will output the **entire** translation sentence, and we use this entire output to compare with the translation true label to calculate loss function

![](images/decoder_output1.png)

## Masks

Because in inference, a word cannot attend to any word after it (as those don't exist yet because we are generating words), so in training, we have to train our decoder so that a word can only attend to previous words

![](images/mask.png)

![](images/mask2.png)

## Output of Decoder for Inference

Similar to Seq2Seq model: predict next word

![](images/decoder_output2.png)

Note: for example, GPT (using Transformer decoder) can only output 512 tokens, so the 513th token won be able to attend to the 1st token anymore.

# Dropout

Everywhere

![](images/dropout1.png)

Note: Dropout is similar to a bagging technique: to 'combine' several submodels (because of dropout) to make predictions. Though we don't average these submodels like random forest, but the big model can learn suitable weights from these submodels combined

![](images/dropout2.png)

# Full model

In [ ]:
# a stack of N=6 identical layers
class Encoder(nn.Module):
    "Core encoder is a stack of N layers"
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, mask):
        "Pass the input (and mask) through each layer in turn."
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

# a stack of N=6 identical layers
class Decoder(nn.Module):
    "Generic N layer decoder with masking."
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

In [ ]:
class EncoderDecoder(nn.Module):
    """
    A standard Encoder-Decoder architecture. Base for this and many 
    other models.
    """
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator # standard linear + softmax generation step, aka blue arrow step 
        
    def forward(self, src, tgt, src_mask, tgt_mask):
        "Take in and process masked src and target sequences."
        return self.decode(self.encode(src, src_mask), src_mask,
                            tgt, tgt_mask)
    
    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

In [ ]:
class Generator(nn.Module):
    "Define standard linear + softmax generation step."
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)

In [ ]:
def make_model(src_vocab, tgt_vocab, N=6, 
               d_model=512, d_ff=2048, h=8, dropout=0.1):
    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), 
                             c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab))
    
    # This was important from their code. 
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform(p)
    return model

# OpenAI GPT

https://lilianweng.github.io/lil-log/2019/01/31/generalized-language-models.html#openai-gpt

Compared to the original transformer architecture, **the transformer decoder model discards the encoder part, so there is only one single input sentence rather than two separate source and target sequences.**

- Building the language model: This model applies multiple transformer blocks over the embeddings of input sequences. Each block contains a masked multi-headed self-attention layer and a pointwise feed-forward layer. **The final output produces a distribution over target tokens after softmax normalization.**
- Use byte pair encoding
- The most substantial upgrade that OpenAI GPT proposed is to **get rid of the task-specific model and use the pre-trained language model directly!**. With similar designs, no customized model structure is needed for other end tasks

![](https://lilianweng.github.io/lil-log/assets/images/OpenAI-GPT-transformer-decoder.png)

## Plugging the classification head

Note that **only the last layer output (after multiple stacked transformers) of the last token (of a sentence) is used for classification task**

![](images/gpt1.png)

- The loss is to minimize the negative log-likelihood for true labels (e.g. classification task). In addition, **adding the LM loss as an auxiliary loss is found to be beneficial**, because:
    - (1) it helps accelerate convergence during training and
    - (2) it is expected to improve the generalization of the supervised model.
    
$$
% <![CDATA[
\begin{aligned}
\mathcal{L}_\text{cls} &= \sum_{(\mathbf{x}, y) \in \mathcal{D}} \log P(y\mid x_1, \dots, x_n) = \sum_{(\mathbf{x}, y) \in \mathcal{D}} \log \text{softmax}(\mathbf{h}_L^{(n)}(\mathbf{x})\mathbf{W}_y) \\
\mathcal{L}_\text{LM} &= -\sum_{i} \log p(x_i\mid x_{i-k}, \dots, x_{i-1}) \\
\mathcal{L} &= \mathcal{L}_\text{cls} + \lambda \mathcal{L}_\text{LM}
\end{aligned} %]]>
$$

In summary:

At the first stage, generative pre-training of a language model can absorb as much free text as possible. Then at the second stage, the model is fine-tuned on specific tasks with a small labeled dataset and a minimal set of new parameters to learn.

## Other tasks?

![](https://lilianweng.github.io/lil-log/assets/images/GPT-downstream-tasks.png)

If the task input contains multiple sentences, a special delimiter token ($) is added between each pair of sentences. The embedding for this delimiter token is a new parameter we need to learn, but it should be pretty minimal.

# BERT

Pre-training of Deep Bidirectional Transformers for Language Understanding

Problem: Language models only use left context or right context, why?
- Reason 1: Directionality is needed to generate a well-formed probability distribution.
- Reason 2: Words can “see themselves” (cheating) in a bidirectional encoder.

**but language understanding is bidirectional.**


Solution: Mask out k% of the input words using a special place holder ``[MASK]``, and then predict the masked words
- They always use k = 15%
                  store          gallon
                    ↑              ↑ 
the man went to the [MASK] to buy a [MASK] of milk
- Too little masking: Too expensive to train
- Too much masking: Not enough context

TODO: can we do a multiclass prediction and choose synonyms of [MASK] words?

## BERT next sentence prediction

![](images/transformer_8.png)

- a binary classification task

## a typical embedding input of BERT

![](images/transformer_9.png)

Note that you will add these 3 embeddings since you will use multi-head encoder (Transformer) which take attention from words (it won't be ideal to concatenate them and perform attention?)

Noted that wordpieces are not simply word embeddings:
- Wordpiece model tokenizes inside words
- BERT uses a variant of the wordpiece model
    - (Relatively) use common words that are in the vocabulary: at, fairfax, 1910s
- Other words are built from wordpieces:
    - hypatia = h ##yp ##ati ##a

## losses

You will have 2 losses: one from predict the [MASK] LM likelihood loss and next sentence prediction loss

**The training loss is the sum of the mean masked LM likelihood and mean next sentence prediction likelihood.**

## Plugging the classification head

Note that the first token of an input sentence is always forced to be [CLS] — a placeholder that will be used later for prediction in downstream tasks.

For classification tasks, we get the prediction by **taking the final hidden state of the special first token** [CLS], $\mathbf{h}^\text{[CLS]}_L$, **and multiplying it with a small weight matrix**, $\text{softmax}(\mathbf{h}^\text{[CLS]}_L \mathbf{W}_\text{cls})$


Bonus for question-answering task:

For QA tasks like SQuAD, we need to predict the text span in the given paragraph for an given question. BERT **predicts two probability distributions of every token**, being the start and the end of the text span. Only two new small matrices, Ws and We, are newly learned during fine-tuning and softmax(h(i)LWs) and softmax(h(i)LWe) define two probability distributions.

## BERT model architecture and training


- Transformer encoder (as before)
- Self-attention ⇒ no locality bias
    - Long-distance context has “equal opportunity”
- Single multiplication per layer ⇒ efficiency on GPU/TPU
- Train on Wikipedia + BookCorpus
- Train 2 model sizes:
    - BERT-Base: 12-layer, 768-hidden, 12-head
    - BERT-Large: 24-layer, 1024-hidden, 16-head
- Trained on 4x4 or 8x8 TPU slice for 4 days

## BERT model finetuning

You can take the model, pre-train, and use that same architecture + weights for different tasks
- Simply learn a classifier built on the top layer for the task that you fine tune for (similarly to ULMfit)

Look at b) for single sentence classification task and c) for QA task (as seen in 'Plugging the classification head' tag above)

![](images/bert1.png)

![](images/transformer_10.png)

# AlBERT

https://lilianweng.github.io/lil-log/2019/01/31/generalized-language-models.html#albert

a light-weighted version of BERT model. An ALBERT model can be trained 1.7x faster with 18x fewer parameters, compared to a BERT model of similar configuration. ALBERT incorporates three changes as follows: the first two help reduce parameters and memory consumption and hence speed up the training speed, while the third one proposes a more chanllenging training task to replace the next sentence prediction (NSP) objective.

1. **Important**: In BERT, the **WordPiece tokenization embedding size E is configured to be the same as the hidden state size H** (TODO: is it in model using Transformer in general too?). That is saying, if we want to increase the model size (larger H), we need to learn a larger tokenization embedding too, which is expensive because it depends on the vocabulary size (V).
==> **tokenization embedding is expected to learn context-independent representation** and the **hidden states are context-dependent**, it makes sense to **separate the size of the hidden layers from the size of vocabulary embedding.** using factorized embedding parameterization (embedding matrix of size V×H is decomposed into two small matrices of size V×E and E×H ???)

2. Cross parameter sharing
3. Sentence order prediction: e.g. Positive sample: two consecutive segments **from the same document**. Negative sample: same as above, but the segment order is switched.